In [28]:
! pip install stop_words

You are using pip version 8.0.3, however version 8.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


In [2]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [3]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words

stopWords = get_stop_words('english')

#Function to make document, word matricies for LDA#
def make_word_matrix(corpus, needToSplit):

    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]

        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
            #Remove stop words#
            text = [word for word in words if word not in stopWords]
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]
            
            #remove stop words
            text = [word for word in words if word not in stopWords]
        
        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [5]:
corpusMatrix = make_word_matrix(docs, 1)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference**

In [6]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  The output of this function are the matricies gamma and phi, where gamma (k vector) is the Dirichlet paramteters and the matrix phi (N x k, where k is the number of topics) are the multinomial paramters.  See page 1004 of paper for derivation.

In [7]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

**Parameter Estimation**

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 for derivation.  

The alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the Mstep() function maximizes for alpha and beta.

In [8]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return alphaNew

In [12]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)



**LDA Function:**
Finally, we implement the entire Latent Dirichlet Allocation method in the LDA function, which takes as its arguments k (the number of topics), D (the number of documents in the corpus), a corpus matrix (the output from make_word_matrix above) and a tolerance (which sets the convergence criteria for the while loops).  For each document d, the function runs until the alpha or beta parameters converge, by first running the E step and then the M step for each document separately.  The final values of phi, gamma, alpha and beta are returned for all D documents in a list.

In [13]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, M, corpusMatrix, tol, needToSplit):

    
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [14]:
LDA(3, 4, corpusMatrix, 0.1, 1)

[[[array([[ 0.19293823,  0.62996044,  0.17710133],
          [ 0.08425836,  0.055018  ,  0.86072363],
          [ 0.23855684,  0.36183222,  0.39961093]]),
   array([[ 0.07578352,  0.03832777,  0.88588871],
          [ 0.24439753,  0.28711716,  0.46848531],
          [ 0.09691595,  0.02645914,  0.87662491],
          [ 0.09691595,  0.02645914,  0.87662491],
          [ 0.09691595,  0.02645914,  0.87662491],
          [ 0.09691595,  0.02645914,  0.87662491]]),
   array([[ 0.33146996,  0.19902686,  0.46950318],
          [ 0.30573183,  0.10192444,  0.59234373],
          [ 0.0886134 ,  0.74997587,  0.16141074]]),
   array([[ 0.2945403 ,  0.11260318,  0.59285652],
          [ 0.08862615,  0.78468572,  0.12668813],
          [ 0.09685719,  0.25239356,  0.65074925],
          [ 0.19583953,  0.15485638,  0.64930409],
          [ 0.07937878,  0.77040765,  0.15021357]])],
  [array([ 2.96624301,  4.97212622,  9.66682441]),
   array([  3.15833444,   4.35659703,  13.09026218]),
   array([ 3.176304

**State of the Union Example**

In order to determine the initial performance time of our algorithm, we implemented LDA on a small corpus of eight documents. These documents are each of the State of the Union Adresses delivered by President Clinton during his time in office. These were found in the nltk Python package.

In [15]:
import nltk
#nltk.download("state_union")
from nltk.corpus import state_union

In [16]:
#creating a diciontary of the state of the Union Addresses for Clinton
fileNames = state_union.fileids()
Clinton = fileNames[50:54]

ClintonSOTU = {}
for i in range(0, len(Clinton)):
    ClintonSOTU[Clinton[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(Clinton[i]))

In [10]:
#should we include the construction of the corpus matrix in LDA function?

In [22]:
%%time
np.random.seed(7)
ClintCorp = make_word_matrix(ClintonSOTU, 0)
Results = LDA(3, 4, ClintCorp, 0.1, 0)

CPU times: user 1min 58s, sys: 248 ms, total: 1min 58s
Wall time: 1min 58s


In [23]:
Results

[[[array([[ 0.51360306,  0.10032805,  0.38606889],
          [ 0.38685442,  0.22932815,  0.38381743],
          [ 0.57678675,  0.28342547,  0.13978778],
          ..., 
          [ 0.48289427,  0.5069549 ,  0.01015082],
          [ 0.31766249,  0.23986816,  0.44246935],
          [ 0.20355186,  0.31191408,  0.48453406]]),
   array([[ 0.51226927,  0.09783816,  0.38989257],
          [ 0.3869699 ,  0.22428603,  0.38874407],
          [ 0.57943024,  0.27838124,  0.14218852],
          ..., 
          [ 0.31759837,  0.23447695,  0.44792467],
          [ 0.20373032,  0.30523251,  0.49103717],
          [ 0.38464187,  0.20025704,  0.4151011 ]]),
   array([[ 0.5077537 ,  0.09855107,  0.39369523],
          [ 0.38278764,  0.22546608,  0.39174629],
          [ 0.57529626,  0.28088506,  0.14381869],
          ..., 
          [ 0.31377038,  0.2354139 ,  0.45081572],
          [ 0.20088637,  0.30586083,  0.4932528 ],
          [ 0.38044585,  0.20129007,  0.41826408]]),
   array([[ 0.51850462,  0.1

**Movie Prediction** 

In addition to text analysis, LDA can be implemented for other problems that have the same structure as a text corpora. In this section, we use LDA in order to predict movies preferred by users of the site MovieLens.com. The goal is to predict another movie that the user likes based on the movies that they have already said that they prefer. The definition of preferred here is that the user rated the movie four or five, where five is the maxiumum possible rating. Users who gave a preferred rating to at least 50 movies were used. A training set of 2,535 users was used to fit the model. The model is then shown all but one of the preferred movies for the users in the test data, and this is used to determine the last movie that the user preferred. Thus, it is possible to evaluate the performance of the model.

In [8]:
#reading in the data
import pandas as pd
trainRatings = pd.read_csv("UserRatingsTraining.csv")
testRatings = pd.read_csv("UserRatingsTest.csv")

In [35]:
#making a dictionary for the training and testing data
trainingUsers = list(set(trainRatings['userID']))
trainingDict = {}
for i in range(0, len(trainingUsers)):
    movies = list(map(str, list(pd.DataFrame(trainRatings[trainRatings['userID'] == trainingUsers[0]])['movieID'])))
    trainingDict[trainingUsers[i]] = movies
    
testUsers = list(set(testRatings['userID']))
testDict = {}
for i in range(0, len(testUsers)):
    movies = list(map(str, list(pd.DataFrame(testRatings[testRatings['userID'] == testUsers[0]])['movieID'])))
    testDict[testUsers[i]] = movies

In [56]:
#running LDA analysis to get parameters for movie training data
movieMatrix = make_word_matrix(trainingDict, 0)
movieOutput = LDA(3, 4, movieMatrix, 0.1, 0)

In [ ]:
#use the above parameters to make predictions for the test set
#hold out one movie randomly, then determine the likelihood of the possible held out movies
#the highest likelihood should be assigned to the last movie, if the model works correctly

## Future Steps

- Compare output to Python package
- Test on corpus in paper
- Model topics on different corpus
- Time and optimize (use Cython, quite slow now)
- Run collaborative filtering
- Compare to Gibbs sampling